In [10]:
import snowflake.connector
import pandas as pd
from tqdm import tqdm

In [11]:
conn = snowflake.connector.Connect(user='ANALYST', password='Fedewap2023!', account='rl43903.east-us-2.azure')

In [19]:
# Edit the filename in read_csv
requests = pd.read_csv("GDPR_Nov_13_GDPR.csv")
emails = requests['From']

In [20]:
# queries a table to see if email is present
def query_table(email, table, column):
    query = f"SELECT COUNT(*) FROM {table} WHERE lower({column}) = '{email}'"
    cur = conn.cursor().execute(query)
    result = cur.fetchone()
    cur.close()
    return result[0] > 0

In [21]:
# Create dictionaries
emails_present = {}
# table-column mapping
tables_columns = {"CHICAGOFIRE_MLS.CF.IQ_CLUB_CRM_FEED_CUSTOMER": "email_address",
          "CHICAGOFIRE_DATABASE.MLS.MASTER_CF": "email_c", 
          "CHICAGOFIRE_DATABASE.SALESFORCE.SF_CONTACT": "email",
          "CHICAGOFIRE_DATABASE.ARCHTICS.CUSTOMERS_2023" : "email_addr"}

In [22]:
# Iterates through the list of emails and makes a list of what emails were found where
for email in tqdm(emails, desc = "Processing Emails"):
    email = email.strip().lower()
    tables_found = []
    for table, column in tables_columns.items():
        if query_table(email, table, column):
            tables_found.append(table)
        if tables_found:
            emails_present[email] = tables_found

Processing Emails: 100%|██████████| 47/47 [00:38<00:00,  1.21it/s]


In [24]:
conn.close()

In [23]:
# Returns emails and where
for email, tables_found_in in emails_present.items():
    print(f"{email} found in tables: {', '.join(tables_found_in)}")

teeleblanc@yahoo.com found in tables: CHICAGOFIRE_MLS.CF.IQ_CLUB_CRM_FEED_CUSTOMER, CHICAGOFIRE_DATABASE.MLS.MASTER_CF, CHICAGOFIRE_DATABASE.SALESFORCE.SF_CONTACT
edake@structured.com found in tables: CHICAGOFIRE_MLS.CF.IQ_CLUB_CRM_FEED_CUSTOMER, CHICAGOFIRE_DATABASE.MLS.MASTER_CF, CHICAGOFIRE_DATABASE.SALESFORCE.SF_CONTACT
francescborrull@gmail.com found in tables: CHICAGOFIRE_MLS.CF.IQ_CLUB_CRM_FEED_CUSTOMER, CHICAGOFIRE_DATABASE.MLS.MASTER_CF, CHICAGOFIRE_DATABASE.SALESFORCE.SF_CONTACT, CHICAGOFIRE_DATABASE.ARCHTICS.CUSTOMERS_2023
manuelvillasis735@yahoo.com found in tables: CHICAGOFIRE_MLS.CF.IQ_CLUB_CRM_FEED_CUSTOMER, CHICAGOFIRE_DATABASE.MLS.MASTER_CF, CHICAGOFIRE_DATABASE.SALESFORCE.SF_CONTACT
peter.keith.107@gmail.com found in tables: CHICAGOFIRE_MLS.CF.IQ_CLUB_CRM_FEED_CUSTOMER, CHICAGOFIRE_DATABASE.MLS.MASTER_CF, CHICAGOFIRE_DATABASE.SALESFORCE.SF_CONTACT
jdmusic@rogers.com found in tables: CHICAGOFIRE_MLS.CF.IQ_CLUB_CRM_FEED_CUSTOMER, CHICAGOFIRE_DATABASE.MLS.MASTER_CF, CHIC